In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import gc

import keras as k
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D

import cv2
from tqdm import tqdm

x_train = []
x_test = []
y_train = []

df_train = pd.read_csv('../input/planets-dataset/planet/planet/train_classes.csv')

flatten = lambda l: [item for sublist in l for item in sublist]
labels = list(set(flatten([l.split(' ') for l in df_train['tags'].values])))

label_map = {l: i for i, l in enumerate(labels)}
inv_label_map = {i: l for l, i in label_map.items()}

for f, tags in tqdm(df_train.values, miniters=1000):
    img = cv2.imread('../input/planets-dataset/planet/planet/train-jpg/{}.jpg'.format(f))
    targets = np.zeros(17)
    for t in tags.split(' '):
        targets[label_map[t]] = 1 
    x_train.append(cv2.resize(img, (32, 32)))
    y_train.append(targets)
    
y_train = np.array(y_train, np.uint8)
x_train = np.array(x_train, np.float16) / 255.

print(x_train.shape)
print(y_train.shape)

split = 35000
x_train, x_valid, y_train, y_valid = x_train[:split], x_train[split:], y_train[:split], y_train[split:]

model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=(32, 32, 3)))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(17, activation='sigmoid'))

model.compile(loss='binary_crossentropy', # We NEED binary here, since categorical_crossentropy l1 norms the output before calculating loss.
              optimizer='adam',
              metrics=['accuracy'])
              
model.fit(x_train, y_train,
          batch_size=128,
          epochs=4,
          verbose=1,
          validation_data=(x_valid, y_valid))
          
from sklearn.metrics import fbeta_score

p_valid = model.predict(x_valid, batch_size=128)
print(y_valid)
print(p_valid)
print(fbeta_score(y_valid, np.array(p_valid) > 0.2, beta=2, average='samples'))



100%|██████████| 40479/40479 [01:33<00:00, 434.03it/s]


(40479, 32, 32, 3)
(40479, 17)
Epoch 1/4
274/274 [==============================] - 66s 242ms/step - loss: 0.2525 - accuracy: 0.4739 - val_loss: 0.1936 - val_accuracy: 0.5101
Epoch 2/4
274/274 [==============================] - 66s 240ms/step - loss: 0.2004 - accuracy: 0.5074 - val_loss: 0.1762 - val_accuracy: 0.5103
Epoch 3/4
274/274 [==============================] - 66s 239ms/step - loss: 0.1883 - accuracy: 0.5056 - val_loss: 0.1702 - val_accuracy: 0.5105
Epoch 4/4
274/274 [==============================] - 66s 242ms/step - loss: 0.1801 - accuracy: 0.5008 - val_loss: 0.1682 - val_accuracy: 0.5145
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 1]
 [0 0 0 ... 0 0 1]
 ...
 [0 0 0 ... 0 0 1]
 [0 0 0 ... 0 0 1]
 [0 0 0 ... 0 0 0]]
[[1.0610789e-02 1.3450682e-03 4.9269408e-02 ... 4.2889118e-03
  7.4422359e-03 4.4797552e-01]
 [1.5549421e-02 1.9000769e-03 1.0949904e-01 ... 1.4909893e-02
  1.0289550e-03 7.3970938e-01]
 [2.3490191e-03 1.1572540e-03 2.2695333e-02 ... 1.4829040e-03
  1.1670887e-03 9.5214784

TypeError: Index(...) must be called with a collection of some kind, 'agriculture cultivation partly_cloudy primary' was passed